# Extracción de datos

## Imports para extracción

In [61]:
import pandas as pd 
import numpy as np
import oandapy as opy
import psycopg2
import logging
import os
from datetime import datetime as dt
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

Extraemos datos de precios (Oanda) para el instrumento que elijamos y de twitter (Base de datos de Prophets)

### Oanda

In [30]:
pricediff = True # Precios o differencia de precios
candleformat = 'midpoint' # ['midpoint', 'bidask']
instrument = 'USD_JPY'
granularity = 'M5'
d1 = '2008-01-01'
min_window = 5
step = int(60/min_window)
d2 = str(dt.now())[:10]
oanda = opy.API(environment='live')

In [39]:
# Extraemos datos cada 2 días (por simplicidad)

dates = pd.date_range(start=d1, end=d2, freq='2D')
df = pd.DataFrame()

print('Se van a descargar {} días'.format(2*len(dates)))
print('Descargando:')

pbar = tqdm(total=len(dates) - 1)

for i in range(0, len(dates) - 1):
    # Oanda toma las fechas en este formato
    d1 = str(dates[i]).replace(' ', 'T')
    d2 = str(dates[i+1]).replace(' ', 'T')
    try:
        # Concatenamos cada día en el dataframe
        data = oanda.get_history(instrument=instrument, 
                                 candleFormat=candleformat,
                                 start=d1, 
                                 end=d2, 
                                 granularity=granularity)
        df = df.append(pd.DataFrame(data['candles']))
        pbar.update(1)
    except:
        pass
pbar.close()
date = pd.DatetimeIndex(df['time'], tz='UTC')
df['date'] = date

Se van a descargar 1979 días
Descargando:


HBox(children=(IntProgress(value=0, max=1978), HTML(value='')))

In [40]:
drops = ['complete', 'time']
df = df.drop(drops, axis=1)
df = df[100:] # Falla en API
df = df.reset_index(drop=True)

In [42]:
if candleformat == 'bidask':
    
    if pricediff:

        prices = [j for j in df.columns if j not in ['date', 'volume']]
        for i in prices:
            df['diff ' + i] = df[i] - df[i].shift(1)
        df = df.drop(prices, axis=1)

        open_bid = ['diff openBid' + str(min_window*(i+1)) for i in range(step)]
        open_ask = ['diff openAsk' + str(min_window*(i+1)) for i in range(step)]
        close_bid = ['diff closeBid' + str(min_window*(i+1)) for i in range(step)]
        close_ask = ['diff closeAsk' + str(min_window*(i+1)) for i in range(step)]
        low_bid = ['diff lowBid' + str(min_window*(i+1)) for i in range(step)]
        low_ask = ['diff lowAsk' + str(min_window*(i+1)) for i in range(step)]
        high_bid = ['diff highBid' + str(min_window*(i+1)) for i in range(step)]
        high_ask = ['diff highAsk' + str(min_window*(i+1)) for i in range(step)]
        volume = ['volume' + str(min_window*(i+1)) for i in range(step)]

        shifts = list(range(1,step+1))

        for v, ob, oa, cb, ca, lb, la, hb, ha, s in zip(volume,
                                                     open_bid, 
                                                     open_ask, 
                                                     close_bid, 
                                                     close_ask, 
                                                     low_bid, 
                                                     low_ask, 
                                                     high_bid, 
                                                     high_ask, 
                                                     shifts):
            df[v] = df['volume'].shift(s)
            df[ob] = df['diff openBid'].shift(s)
            df[oa] = df['diff openAsk'].shift(s)
            df[cb] = df['diff closeBid'].shift(s)
            df[ca] = df['diff closeAsk'].shift(s)
            df[lb] = df['diff lowBid'].shift(s)
            df[la] = df['diff lowAsk'].shift(s)
            df[hb] = df['diff highBid'].shift(s)
            df[ha] = df['diff highAsk'].shift(s)

    else:

        open_bid = ['openBid' + str(min_window*(i+1)) for i in range(step)]
        open_ask = ['openAsk' + str(min_window*(i+1)) for i in range(step)]
        close_bid = ['closeBid' + str(min_window*(i+1)) for i in range(step)]
        close_ask = ['closeAsk' + str(min_window*(i+1)) for i in range(step)]
        low_bid = ['lowBid' + str(min_window*(i+1)) for i in range(step)]
        low_ask = ['lowAsk' + str(min_window*(i+1)) for i in range(step)]
        high_bid = ['highBid' + str(min_window*(i+1)) for i in range(step)]
        high_ask = ['highAsk' + str(min_window*(i+1)) for i in range(step)]
        volume = ['volume' + str(min_window*(i+1)) for i in range(step)]

        shifts = list(range(1,step+1))

        for v, ob, oa, cb, ca, lb, la, hb, ha, s in zip(volume,
                                                     open_bid, 
                                                     open_ask, 
                                                     close_bid, 
                                                     close_ask, 
                                                     low_bid, 
                                                     low_ask, 
                                                     high_bid, 
                                                     high_ask, 
                                                     shifts):
            df[v] = df['volume'].shift(s)
            df[ob] = df['openBid'].shift(s)
            df[oa] = df['openAsk'].shift(s)
            df[cb] = df['closeBid'].shift(s)
            df[ca] = df['closeAsk'].shift(s)
            df[lb] = df['lowBid'].shift(s)
            df[la] = df['lowAsk'].shift(s)
            df[hb] = df['highBid'].shift(s)
            df[ha] = df['highAsk'].shift(s)
else:  
    if pricediff:

        prices = [j for j in df.columns if j not in ['date', 'volume']]
        for i in prices:
            df['diff ' + i] = df[i] - df[i].shift(1)
        df = df.drop(prices, axis=1)

        open_ = ['diff openMid' + str(min_window*(i+1)) for i in range(step)]
        close = ['diff closeMid' + str(min_window*(i+1)) for i in range(step)]
        low = ['diff lowMid' + str(min_window*(i+1)) for i in range(step)]
        high = ['diff highMid' + str(min_window*(i+1)) for i in range(step)]
        volume = ['volume' + str(min_window*(i+1)) for i in range(step)]

        shifts = list(range(1,step+1))

        for v, o, c, l, h, s in zip(volume,
                                     open_, 
                                     close, 
                                     low, 
                                     high, 
                                     shifts):
            df[v] = df['volume'].shift(s)
            df[o] = df['diff openMid'].shift(s)
            df[c] = df['diff closeMid'].shift(s)
            df[l] = df['diff lowMid'].shift(s)
            df[h] = df['diff highMid'].shift(s)

    else:

        open_ = ['openMid' + str(min_window*(i+1)) for i in range(step)]
        close = ['closeMid' + str(min_window*(i+1)) for i in range(step)]
        low = ['lowMid' + str(min_window*(i+1)) for i in range(step)]
        high = ['highMid' + str(min_window*(i+1)) for i in range(step)]
        volume = ['volume' + str(min_window*(i+1)) for i in range(step)]

        shifts = list(range(1,step+1))

        for v, o, c, l, h, s in zip(volume,
                                     open_, 
                                     close, 
                                     low, 
                                     high, 
                                     shifts):
            df[v] = df['volume'].shift(s)
            df[o] = df['openMid'].shift(s)
            df[c] = df['closeMid'].shift(s)
            df[l] = df['lowMid'].shift(s)
            df[h] = df['highMid'].shift(s)


In [44]:
df = df[step+1:]

In [46]:
df['date'] = df['date'].astype(str)
df['d2'] = df['date'].str[14:]
df = df[df['d2'] == '00:00+00:00']

/home/usuario-rtd/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/usuario-rtd/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [50]:
df = df.reset_index(drop=True)
df = df.drop('d2', axis=1)

In [52]:
fake_drop = volume + ['volume', 'date']
df['H'] = df.drop(fake_drop, 1).max(axis=1)
df['L'] = df.drop(fake_drop, 1).min(axis=1)
df['vol'] = df[volume].sum(axis=1)

In [57]:
df['date'] = df['date'].str[:13]
pr = df.copy()

### Twitter

In [56]:
def db_conn():
    """
    Método que hace la conexión a la base de datos
    Args:
        conn_creds(dict): diccionario donde vienen las credenciales de
        la conexión a la base de datos
                         host(str): host que hospeda a la base de datos
                         port(str): puerto donde está disponible la base de datos
                         user(str): usuario con el que se hará la conexión
                         password(str): contraseña del usuario en la BD
                         database(str): nombre de la base de datos
    Returns:
        conn: objeto que contiene la sesión de una conexión a la BD
    """
    try:
        conn = psycopg2.connect(
            host=os.environ['ID_HOST'],
            port=os.environ['ID_PORT'],
            user=os.environ['ID_USER'],
            password=os.environ['ID_PASSWORD'],
            database=os.environ['ID_DB'],
        )
    except Exception as error:
        logging.error(error)

    return conn

def download(conn, query):
    """
    Descarga datos de la base de datos según la consulta insertada
    Args:
        conn (connection): objeto que contiene la sesión de una
                           conexión a la base de datos
        query (str): String donde se define el query a ejecutarse
    Returns:
        df (DataFrame): Tabla con los datos que elegimos
    """
    try:
        df = pd.read_sql(query, conn)
        conn.commit()
    finally:
        conn.close()


    return df

In [58]:
conn = db_conn()
query = '''SELECT date_trunc('hour',cast(date as timestamp)), avg(polarity) polarity, avg(subjectivity) subjectivity
           FROM tweets GROUP BY date_trunc('hour',cast(date as timestamp))'''
tw = download(conn, query)

In [62]:
tw['date_trunc'] = tw['date_trunc'] + timedelta(hours=5)
tw['date_trunc'] = tw['date_trunc'].astype(str)
tw['date_trunc'] = tw['date_trunc'].str[:13]

In [64]:
df = pd.merge(pr, tw, left_on='date', right_on='date_trunc', how='left')
df = df.drop(['date', 'date_trunc'], axis=1)

In [66]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

# Modelos

## Imports para modelos

In [77]:
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier, TPOTRegressor

In [69]:
def train_test(df, response, train_size=0.75, time_series=False, scaling=None):
    """
    Regresa train y test sets

    Args:
        df (DataFrame): Datos listos para el modelo
        response (str): Variable respuesta
        train_size (float): % Train Size
        time_series (boolean): Si es serie de tiempo o no
        scaling (str): ['standard', 'minmax', 'maxabs', 'robust', 'quantile']
    Returns:
        X_train (Array): conjunto de datos de entrenamiento (indep)
        X_test (Array): conjunto de datos de prueba (indep)
        y_train (Array): conjunto de datos de entrenamiento (dep)
        y_test (Array): conjunto de datos de prueba (dep)
    """

    data = df.copy()
    X = data.drop(response, 1)
    y = data[response]

    logging.info('X columns')
    logging.info(list(X.columns))
    logging.info('Response')
    logging.info(response)

    if time_series:
        trainsize = int(train_size*len(X))
        X_train = X[:trainsize].values
        X_test = X[trainsize:].values
        y_train = y[:trainsize].values
        y_test = y[trainsize:].values

    else:
        X_train, X_test, y_train, y_test = train_test_split(X.values,
                                                            y.values,
                                                            random_state=0,
                                                            train_size=train_size)
    if scaling == 'standard':
        scaler = preprocessing.StandardScaler()
    if scaling == 'minmax':
        scaler = preprocessing.MinMaxScaler()
    if scaling == 'maxabs':
        scaler = preprocessing.MaxAbsScaler()
    if scaling == 'robust':
        scaler = preprocessing.RobustScaler()
    if scaling == 'quantile':
        scaler = preprocessing.QuantileTransformer()

    if scaling != None:
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

#### Train-Test

In [72]:
if pricediff:
    response = 'future diff close'
    actual = 'diff closeMid'
else:
    response = 'future close'
    actual = 'closeMid'
    
df[response] = df[actual].shift(-1)

In [74]:
X_train, X_test, y_train, y_test = train_test(df, response, train_size=0.75, time_series=True, scaling=None)

In [ ]:
# Agregar MSE
# Agregar loop para todas las responses y todos los modelos

### TPOT

In [78]:
def tpotreg(X_train, y_train):
    """
    Usando TPOT (Tree-Based Pipeline Optimization Tool), librería de AutoML,
    genera el "mejor" modelo de regresión automáticamente
    Args:
        X_train (Array): conjunto de datos de entrenamiento (regresores)
        y_train (Array): conjunto de datos de entrenamiento (objetivo)
    Returns:
        tpotmod (modelo): Modelo de regresión generado con TPOT
    """

    pipeline_optimizer = TPOTRegressor(generations=5,
                                       population_size=50,
                                       cv=5,
                                       random_state=42,
                                       verbosity=2,
                                       n_jobs=4)
    tpotmod = pipeline_optimizer.fit(X_train, y_train)

    return tpotmod

In [79]:
tpotmod = tpotreg(X_train, y_train)

/home/usuario-rtd/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Optimization Progress:   6%|▌         | 18/300 [11:38<3:37:39, 46.31s/pipeline]Process ForkPoolWorker-11:
                                                                               



TPOT closed prematurely. Will use the current best pipeline.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(range(len(y_test[:30])),tpotmod.predict(X_test[:50]), color='r', label='Predicted')
plt.plot(range(len(y_test[:30])),y_test[:50], color='b', label='Expected')
plt.legend()
plt.show()